# Example of the SASRec training and inference stages
Note that all the given examples can be run without using PySpark, using only Pandas

In [1]:
import lightning as L
from lightning.pytorch.loggers import CSVLogger
from lightning.pytorch.callbacks import ModelCheckpoint
from torch.utils.data import DataLoader
import torch

from replay.metrics import OfflineMetrics, Recall, Precision, MAP, NDCG, HitRate, MRR
from replay.metrics.torch_metrics_builder import metrics_to_df
from replay.splitters import LastNSplitter
from replay.data import (
    FeatureHint,
    FeatureInfo,
    FeatureSchema,
    FeatureSource,
    FeatureType,
    Dataset,
)
from replay.models.nn.optimizer_utils import FatOptimizerFactory
from replay.models.nn.sequential.callbacks import (
    ValidationMetricsCallback,
    SparkPredictionCallback,
    PandasPredictionCallback, 
    TorchPredictionCallback,
    QueryEmbeddingsPredictionCallback
)
from replay.models.nn.sequential.postprocessors import RemoveSeenItems
from replay.data.nn import (
    SequenceTokenizer,
    SequentialDataset,
    TensorFeatureSource,
    TensorSchema,
    TensorFeatureInfo
)
from replay.models.nn.sequential import SasRec
from replay.models.nn.sequential.sasrec import (
    SasRecPredictionDataset,
    SasRecTrainingDataset,
    SasRecValidationDataset,
    SasRecPredictionBatch,
    SasRecModel
)

import pandas as pd
import polars as pl

## Prepare data
### Load raw movielens-1M interactions, item features and user features.
In the current implementation, the SASRec does not take into account the features of items or users. They are only used to get a complete list of users and items.

In [2]:
!pip install rs-datasets

In [3]:
from rs_datasets import MovieLens

In [4]:
movielens = MovieLens("1m")
interactions = movielens.ratings
user_features = movielens.users
item_features = movielens.items

In [5]:
import numpy as np
from replay.preprocessing.filters import NumInteractionsFilter

In [6]:
interactions = NumInteractionsFilter(num_interactions=5).transform(interactions)
sz = interactions.shape[0]
cols = 8
interactions["int_cat_list"] = np.random.randint(0, 10, size=(sz, cols)).tolist()
interactions["int_num_list"] = np.random.randn(sz, cols).tolist()
interactions["int_cat"] = np.random.randint(0, 10, size=sz)
interactions["int_num"] = np.random.randn(sz)
# interactions = pl.DataFrame(interactions)

In [7]:
interactions

,user_id,item_id,rating,timestamp,int_cat_list,int_num_list,int_cat,int_num
22,1,1270,5,978300055,"[8, 7, 2, 8, 8, 9, 3, 8]","[0.24572880036476222, -0.7152729663680886, 0.1...",0,0.440003
24,1,2340,3,978300103,"[3, 2, 4, 1, 4, 1, 3, 4]","[-2.498021523672135, 1.734690788910874, 0.5201...",5,0.280106
27,1,1721,4,978300055,"[5, 4, 1, 8, 8, 8, 2, 6]","[-0.03397852975629857, -0.5360900145700243, -0...",1,-1.040534
31,1,3186,4,978300019,"[4, 1, 4, 5, 0, 0, 8, 6]","[-0.0028166290648475112, 0.35102236034715734, ...",5,1.275206
37,1,1022,5,978300055,"[9, 8, 9, 3, 0, 1, 0, 9]","[-0.9379731225884952, -0.6439550539955673, 0.9...",3,-1.627082
...,...,...,...,...,...,...,...,...
999873,6040,593,5,956703954,"[2, 5, 3, 3, 2, 7, 4, 2]","[-1.4808639701602377, -1.8268327190500286, 0.4...",9,-0.569295
1000007,6040,1961,4,956703977,"[2, 5, 3, 0, 8, 0, 4, 7]","[-0.4054356535467929, -1.7856277442760993, -0....",2,1.434125
1000138,6040,858,4,956703932,"[4, 9, 9, 1, 8, 7, 9, 6]","[1.1419282800378616, 0.2226156482048346, -1.09...",2,-1.084541
1000153,6040,2384,4,956703954,"[1, 8, 2, 1, 4, 9, 3, 6]","[0.8937552446897207, 0.5148946738816954, 0.946...",5,-0.246546


In [8]:
sz = user_features.shape[0]
cols = 6
user_features["user_cat_list"] = np.random.randint(0, 10, size=(sz, cols)).tolist()
user_features["user_num_list"] = np.random.randn(sz, cols).tolist()
user_features["user_cat"] = np.random.randint(0, 10, size=sz)
user_features["user_num"] = np.random.randn(sz)
# user_features = pl.DataFrame(user_features)
user_features

,user_id,gender,age,occupation,zip_code,user_cat_list,user_num_list,user_cat,user_num
0,1,F,1,10,48067,"[9, 3, 0, 3, 6, 3]","[1.2111897298669003, -0.7922902667218017, -0.9...",0,1.997819
1,2,M,56,16,70072,"[8, 3, 9, 2, 4, 2]","[0.18382585047496333, -0.7058809264180206, 1.0...",0,-0.371579
2,3,M,25,15,55117,"[0, 9, 0, 6, 7, 3]","[0.795785306692731, -1.3811366620986492, -0.70...",3,1.095353
3,4,M,45,7,02460,"[9, 6, 9, 9, 2, 1]","[0.03285482950274071, -0.8895047563145416, -2....",5,-1.527685
4,5,M,25,20,55455,"[7, 4, 7, 7, 2, 6]","[-1.687836420724842, 1.2224750458654166, -0.39...",2,-1.859056
...,...,...,...,...,...,...,...,...,...
6035,6036,F,25,15,32603,"[4, 5, 8, 8, 1, 7]","[-1.4000342842986702, 0.26085636617977365, 0.5...",5,-0.796250
6036,6037,F,45,1,76006,"[3, 1, 4, 1, 2, 4]","[0.09984811373233482, 0.19913765943789327, -0....",0,-0.136610
6037,6038,F,56,1,14706,"[4, 2, 5, 1, 7, 6]","[1.719147802279268, -0.09181037467022449, -0.1...",6,-0.855963
6038,6039,F,45,0,01060,"[0, 3, 5, 1, 4, 6]","[-0.359620324202194, -0.42374874936116047, -0....",4,-0.546370


In [9]:
sz = item_features.shape[0]
cols = 13
item_features.sort_values("item_id", inplace=True)
item_features["item_cat_list"] = np.random.randint(0, 10, size=(sz, cols)).tolist()
item_features["item_num_list"] = np.random.randn(sz, cols).tolist()
item_features["item_cat"] = np.random.randint(0, 10, size=sz).tolist()
item_features["item_num"] = np.random.randn(sz).tolist()
item_features["genres"] = item_features["genres"].str.split('|')
# item_features = pl.DataFrame(item_features)
item_features

,item_id,title,genres,item_cat_list,item_num_list,item_cat,item_num
0,1,Toy Story (1995),"[Animation, Children's, Comedy]","[1, 2, 9, 7, 0, 8, 5, 7, 1, 9, 0, 0, 0]","[1.483023156832867, 0.2017158849271806, -1.072...",9,0.100583
1,2,Jumanji (1995),"[Adventure, Children's, Fantasy]","[4, 1, 8, 3, 8, 7, 2, 6, 6, 2, 3, 3, 3]","[0.5782075484621079, -0.21321734323060212, 1.3...",3,1.175274
2,3,Grumpier Old Men (1995),"[Comedy, Romance]","[9, 2, 4, 6, 5, 0, 6, 8, 1, 1, 4, 4, 2]","[-1.1167608107588163, -0.41245990294713114, 0....",4,-0.155622
3,4,Waiting to Exhale (1995),"[Comedy, Drama]","[9, 7, 1, 2, 5, 6, 1, 3, 5, 3, 1, 2, 5]","[0.4794109226302443, -1.092724901461659, -0.35...",4,-0.466517
4,5,Father of the Bride Part II (1995),[Comedy],"[4, 1, 9, 6, 1, 8, 6, 4, 8, 7, 5, 9, 1]","[-0.9490019410443985, -0.09912696671171735, -0...",5,1.352423
...,...,...,...,...,...,...,...
3878,3948,Meet the Parents (2000),[Comedy],"[9, 3, 0, 2, 5, 2, 4, 1, 3, 2, 5, 2, 7]","[-0.12663426919750576, -1.1929886761016977, 0....",7,0.377766
3879,3949,Requiem for a Dream (2000),[Drama],"[1, 1, 9, 7, 0, 8, 7, 0, 1, 5, 4, 8, 1]","[-0.9899699214818928, 0.7804655494279595, 0.41...",4,-0.656263
3880,3950,Tigerland (2000),[Drama],"[8, 2, 4, 0, 5, 4, 1, 3, 7, 0, 9, 2, 4]","[0.009821626086104252, -1.379736875660516, -0....",5,1.077170
3881,3951,Two Family House (2000),[Drama],"[7, 1, 6, 6, 6, 3, 0, 5, 9, 6, 6, 1, 7]","[0.3219107308272653, 0.41538625133745893, -0.4...",0,0.436232


Removing duplicates in the timestamp column without changing the original items order where timestamp is the same

In [10]:
# interactions["timestamp"] = interactions["timestamp"].astype("int64")
# interactions = interactions.sort_values(by="timestamp")
# interactions["timestamp"] = interactions.groupby("user_id").cumcount()

### Split interactions into the train, validation and test datasets using LastNSplitter

In [11]:
splitter = LastNSplitter(
    N=1,
    divide_column="user_id",
    query_column="user_id",
    strategy="interactions",
)

raw_test_events, raw_test_gt = splitter.split(interactions)
raw_validation_events, raw_validation_gt = splitter.split(raw_test_events)
raw_train_events = raw_validation_events

### Prepare FeatureSchema required to create Dataset

In [12]:
def prepare_feature_schema(is_ground_truth: bool) -> FeatureSchema:
    all_features = FeatureSchema(
        [
            FeatureInfo(
                column="user_id",
                feature_hint=FeatureHint.QUERY_ID,
                feature_type=FeatureType.CATEGORICAL,
            ),
            FeatureInfo(
                column="item_id",
                feature_hint=FeatureHint.ITEM_ID,
                feature_type=FeatureType.CATEGORICAL,
            ),
            FeatureInfo(
                column="int_cat_list",
                feature_type=FeatureType.CATEGORICAL_LIST,
            ),
            FeatureInfo(
                column="int_num_list",
                feature_type=FeatureType.NUMERICAL_LIST,
            ),
            FeatureInfo(
                column="int_cat",
                feature_type=FeatureType.CATEGORICAL,
            ),
            FeatureInfo(
                column="int_num",
                feature_type=FeatureType.NUMERICAL,
            ),
        ]
    )
    if is_ground_truth:
        return all_features

    all_features = all_features + FeatureSchema(
        [
            FeatureInfo(
                column="timestamp",
                feature_type=FeatureType.NUMERICAL,
                feature_hint=FeatureHint.TIMESTAMP,
            ),
            # item features
            FeatureInfo(
                column="item_cat_list",
                feature_type=FeatureType.CATEGORICAL_LIST,
                feature_source=FeatureSource.ITEM_FEATURES,
            ),
            FeatureInfo(
                column="genres",
                feature_type=FeatureType.CATEGORICAL_LIST,
                feature_source=FeatureSource.ITEM_FEATURES,
            ),
            FeatureInfo(
                column="item_num_list",
                feature_type=FeatureType.NUMERICAL_LIST,
                feature_source=FeatureSource.ITEM_FEATURES,
            ),
            FeatureInfo(
                column="item_cat",
                feature_type=FeatureType.CATEGORICAL,
                feature_source=FeatureSource.ITEM_FEATURES,
            ),
            FeatureInfo(
                column="item_num",
                feature_type=FeatureType.NUMERICAL,
                feature_source=FeatureSource.ITEM_FEATURES,
            ),
            # query features
            FeatureInfo(
                column="user_cat_list",
                feature_type=FeatureType.CATEGORICAL_LIST,
                feature_source=FeatureSource.QUERY_FEATURES,
            ),
            FeatureInfo(
                column="user_num_list",
                feature_type=FeatureType.NUMERICAL_LIST,
                feature_source=FeatureSource.QUERY_FEATURES,
            ),
            FeatureInfo(
                column="user_cat",
                feature_type=FeatureType.CATEGORICAL,
                feature_source=FeatureSource.QUERY_FEATURES,
            ),
            FeatureInfo(
                column="user_num",
                feature_type=FeatureType.NUMERICAL,
                feature_source=FeatureSource.QUERY_FEATURES,
            ),
        ]
    )
    return all_features

### Create Dataset for the training stage

In [13]:
train_dataset = Dataset(
    feature_schema=prepare_feature_schema(is_ground_truth=False),
    interactions=raw_train_events,
    query_features=user_features,
    item_features=item_features,
    check_consistency=True,
    categorical_encoded=False,
)

### Create Datasets (events and ground_truth) for the validation stage

In [14]:
validation_dataset = Dataset(
    feature_schema=prepare_feature_schema(is_ground_truth=False),
    interactions=raw_validation_events,
    query_features=user_features,
    item_features=item_features,
    check_consistency=True,
    categorical_encoded=False,
)
validation_gt = Dataset(
    feature_schema=prepare_feature_schema(is_ground_truth=True),
    interactions=raw_validation_gt,
    check_consistency=True,
    categorical_encoded=False,
)

### Create Datasets (events and ground_truth) for the testing stage

In [15]:
test_dataset = Dataset(
    feature_schema=prepare_feature_schema(is_ground_truth=False),
    interactions=raw_test_events,
    query_features=user_features,
    item_features=item_features,
    check_consistency=True,
    categorical_encoded=False,
)
test_gt = Dataset(
    feature_schema=prepare_feature_schema(is_ground_truth=True),
    interactions=raw_test_gt,
    check_consistency=True,
    categorical_encoded=False,
)

### Create the tensor schema
A schema shows the correspondence of columns from the source dataset with the internal representation of tensors inside the model

In [16]:
ITEM_FEATURE_NAME = "item_id_seq"

tensor_schema = TensorSchema([
    TensorFeatureInfo(
        name=ITEM_FEATURE_NAME,
        is_seq=True,
        feature_type=FeatureType.CATEGORICAL,
        feature_source=TensorFeatureSource(FeatureSource.INTERACTIONS, train_dataset.feature_schema.item_id_column),
        feature_hint=FeatureHint.ITEM_ID,
    ),
    TensorFeatureInfo(
        name="timestamp_seq",
        is_seq=True,
        feature_type=FeatureType.NUMERICAL,
        feature_source=TensorFeatureSource(FeatureSource.INTERACTIONS, "timestamp"),
        feature_hint=FeatureHint.TIMESTAMP,
    ),
    TensorFeatureInfo(
        name="item_cat_list_seq",
        is_seq=True,
        feature_type=FeatureType.CATEGORICAL_LIST,
        feature_source=TensorFeatureSource(FeatureSource.ITEM_FEATURES, "item_cat_list"),
    ),
    TensorFeatureInfo(
        name="genres_seq",
        is_seq=True,
        feature_type=FeatureType.CATEGORICAL_LIST,
        feature_source=TensorFeatureSource(FeatureSource.ITEM_FEATURES, "genres"),
    ),
    TensorFeatureInfo(
        name="item_num_list_seq",
        is_seq=True,
        feature_type=FeatureType.NUMERICAL_LIST,
        feature_source=TensorFeatureSource(FeatureSource.ITEM_FEATURES, "item_num_list"),
    ),
    TensorFeatureInfo(
        name="item_cat_seq",
        is_seq=True,
        feature_type=FeatureType.CATEGORICAL,
        feature_source=TensorFeatureSource(FeatureSource.ITEM_FEATURES, "item_cat"),
    ),
    TensorFeatureInfo(
        name="item_num_seq",
        is_seq=True,
        feature_type=FeatureType.NUMERICAL,
        feature_source=TensorFeatureSource(FeatureSource.ITEM_FEATURES, "item_num"),
    ),
    TensorFeatureInfo(
        name="int_cat_list_seq",
        is_seq=True,
        feature_type=FeatureType.CATEGORICAL_LIST,
        feature_source=TensorFeatureSource(FeatureSource.INTERACTIONS, "int_cat_list"),
    ),
    TensorFeatureInfo(
        name="int_num_list_seq",
        is_seq=True,
        feature_type=FeatureType.NUMERICAL_LIST,
        feature_source=TensorFeatureSource(FeatureSource.INTERACTIONS, "int_num_list"),
    ),
    TensorFeatureInfo(
        name="int_cat_seq",
        is_seq=True,
        feature_type=FeatureType.CATEGORICAL,
        feature_source=TensorFeatureSource(FeatureSource.INTERACTIONS, "int_cat"),
    ),
    TensorFeatureInfo(
        name="int_num_seq",
        is_seq=True,
        feature_type=FeatureType.NUMERICAL,
        feature_source=TensorFeatureSource(FeatureSource.INTERACTIONS, "int_num"),
    ),
    TensorFeatureInfo(
        name="user_cat_list_seq",
        is_seq=True,
        feature_type=FeatureType.CATEGORICAL_LIST,
        feature_source=TensorFeatureSource(FeatureSource.QUERY_FEATURES, "user_cat_list"),
    ),
    TensorFeatureInfo(
        name="user_cat_list_not_seq",
        is_seq=False,
        feature_type=FeatureType.CATEGORICAL_LIST,
        feature_source=TensorFeatureSource(FeatureSource.QUERY_FEATURES, "user_cat_list"),
    ),
    TensorFeatureInfo(
        name="user_cat_seq",
        is_seq=True,
        feature_type=FeatureType.CATEGORICAL,
        feature_source=TensorFeatureSource(FeatureSource.QUERY_FEATURES, "user_cat"),
    ),
    TensorFeatureInfo(
        name="user_cat_not_seq",
        is_seq=False,
        feature_type=FeatureType.CATEGORICAL,
        feature_source=TensorFeatureSource(FeatureSource.QUERY_FEATURES, "user_cat"),
    ),
])

### Create sequential datasets using SequenceTokenizer
The SequentialDataset internally store data in the form of sequences of items sorted by increasing interaction time (timestamp). A SequenceTokenizer is used to convert to this format. In addition, the SequenceTokenizer encodes all categorical columns from the source dataset and stores mapping inside itself.
SequentialDataset.keep_common_query_ids is used to leave only sequences from the same users

In [17]:
tokenizer = SequenceTokenizer(tensor_schema, allow_collect_to_master=True)
tokenizer.fit(train_dataset)

sequential_train_dataset = tokenizer.transform(train_dataset)

sequential_validation_dataset = tokenizer.transform(validation_dataset)
sequential_validation_gt = tokenizer.transform(validation_gt, [tensor_schema.item_id_feature_name])

sequential_validation_dataset, sequential_validation_gt = SequentialDataset.keep_common_query_ids(
    sequential_validation_dataset, sequential_validation_gt
)

In [18]:
sequential_train_dataset._sequences

,item_id_seq,timestamp_seq,item_cat_list_seq,genres_seq,item_num_list_seq,item_cat_seq,item_num_seq,int_cat_list_seq,int_num_list_seq,int_cat_seq,int_num_seq,user_cat_list_seq,user_cat_list_not_seq,user_cat_seq,user_cat_not_seq
user_id,,,,,,,,,,,,,,,
0,"[3117, 1672, 1250]","[978300019, 978300055, 978300055]","[[5, 0, 4, 9, 9, 5, 7, 7, 3, 7, 8, 2, 1], [2, ...","[[6], [6, 5], [2, 11]]","[[-0.7371095169935106, 0.5172784209909229, -1....","[9, 4, 5]","[[0.3400542192326083], [-0.5327000480128626], ...","[[0, 1, 0, 2, 3, 3, 4, 5], [2, 0, 1, 4, 4, 4, ...","[[-0.0028166290648475112, 0.35102236034715734,...","[0, 2, 1]","[[1.2752063911674782], [-1.040534072536861], [...","[[0, 1, 2, 1, 3, 1], [0, 1, 2, 1, 3, 1], [0, 1...","[0, 1, 2, 1, 3, 1]","[0, 0, 0]",[0]
1,"[1180, 1199, 1192]","[978298124, 978298151, 978298151]","[[5, 2, 8, 5, 5, 6, 1, 5, 7, 5, 9, 6, 5], [0, ...","[[7, 3], [6, 13], [7, 3, 5, 11, 13]]","[[-0.6719777117797717, -0.15292580017366167, -...","[4, 5, 5]","[[0.07143264876231152], [0.2492790165508092], ...","[[7, 8, 4, 5, 2, 1, 3, 5], [4, 8, 6, 4, 0, 9, ...","[[0.021013318399510088, 0.649640590328758, -0....","[3, 4, 4]","[[1.0805619729097102], [-0.7668057972668736], ...","[[4, 1, 0, 5, 6, 5], [4, 1, 0, 5, 6, 5], [4, 1...","[4, 1, 0, 5, 6, 5]","[0, 0, 0]",[0]
2,"[589, 2789, 3465]","[978297018, 978297039, 978297068]","[[3, 8, 0, 5, 0, 1, 5, 1, 9, 5, 0, 9, 9], [4, ...","[[6, 9], [2, 6], [2]]","[[0.08949539369713089, 1.4161395609414833, 1.3...","[4, 5, 6]","[[-0.683346597120824], [-0.831939087874874], [...","[[4, 0, 7, 6, 9, 0, 7, 3], [2, 8, 8, 0, 7, 1, ...","[[-0.7979330310055955, 0.01904892113566897, 0....","[5, 3, 5]","[[-0.9717097665967962], [-0.29187845503131743]...","[[2, 0, 2, 3, 7, 1], [2, 0, 2, 3, 7, 1], [2, 0...","[2, 0, 2, 3, 7, 1]","[1, 1, 1]",[1]
3,"[1192, 1081, 3399]","[978293924, 978293964, 978294008]","[[6, 6, 1, 8, 2, 0, 7, 1, 0, 4, 1, 0, 9], [2, ...","[[7, 3, 5, 11, 13], [1, 6, 4, 11], [6]]","[[-0.2387270365605642, -0.8439073598335783, -0...","[5, 5, 0]","[[0.9397320452383293], [0.33043226126353237], ...","[[1, 4, 6, 6, 0, 2, 6, 9], [3, 4, 4, 9, 3, 3, ...","[[1.2986729199803495, -1.1270799714320086, -0....","[4, 6, 3]","[[-0.6018698159177285], [0.24092595217721435],...","[[0, 3, 0, 0, 5, 8], [0, 3, 0, 0, 5, 8], [0, 3...","[0, 3, 0, 0, 5, 8]","[2, 2, 2]",[2]
4,"[896, 907, 2648]","[978241072, 978241072, 978241072]","[[0, 0, 6, 5, 5, 2, 9, 9, 8, 8, 2, 7, 9], [2, ...","[[6, 9], [3, 1, 6, 14], [2, 10]]","[[-1.5867769587025542, -1.0325545345483913, 0....","[5, 0, 2]","[[1.4447708685088008], [-0.5718358307077351], ...","[[4, 1, 0, 7, 6, 2, 9, 1], [8, 5, 6, 3, 6, 6, ...","[[1.0751836701583652, -0.209538781476585, 0.19...","[7, 3, 3]","[[-1.2840729543035985], [-0.7620753702928027],...","[[7, 6, 7, 7, 5, 3], [7, 6, 7, 7, 5, 3], [7, 6...","[7, 6, 7, 7, 5, 3]","[3, 3, 3]",[3]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6035,"[1672, 3369, 2359]","[956709349, 956709350, 956709350]","[[2, 3, 4, 1, 0, 7, 1, 2, 3, 8, 1, 4, 8], [6, ...","[[6, 5], [7, 1, 4], [10, 11]]","[[-1.646148393952946, 0.1432692730406104, -1.8...","[4, 9, 7]","[[-0.5327000480128626], [1.0244708036566967], ...","[[1, 2, 4, 9, 2, 1, 4, 2], [6, 1, 7, 6, 9, 3, ...","[[1.52817633451984, -0.07949328621943147, -0.5...","[0, 3, 2]","[[-1.9064846359939154], [-0.34495441787373565]...","[[6, 9, 4, 4, 8, 7], [6, 9, 4, 4, 8, 7], [6, 9...","[6, 9, 4, 4, 8, 7]","[2, 2, 2]",[2]
6036,"[1813, 693, 3439]","[956708997, 956708997, 956708997]","[[8, 6, 4, 1, 9, 7, 5, 0, 5, 3, 8, 4, 9], [1, ...","[[7, 11], [6], [17]]","[[-0.36946844520740235, -1.6465738035656168, -...","[6, 9, 7]","[[0.780361891121729], [-0.290044401792015], [0...","[[0, 6, 9, 1, 0, 8, 6, 8], [8, 3, 1, 3, 2, 3, ...","[[-0.0763071077550825, -0.4373363711879449, 2....","[0, 0, 6]","[[-1.3163328783677797], [-1.2103231865302504],...","[[1, 8, 6, 8, 5, 6], [1, 8, 6, 8, 5, 6], [1, 8...","[1, 8, 6, 8, 5, 6]","[0, 0, 0]",[0]
6037,"[908, 3327, 1192]","[956706827, 956706827, 956706876]","[[1, 5, 7, 7,

In [ ]:
seq = sequential_train_dataset.get_sequence(0, "item_id_seq")
print(seq.shape)
seq

In [ ]:
seq = sequential_train_dataset.get_sequence(0, "user_cat_not_seq")
print(seq.shape)
seq

In [ ]:
test_query_ids = test_gt.query_ids
test_query_ids_np = tokenizer.query_id_encoder.transform(test_query_ids)["user_id"].to_numpy()
sequential_test_dataset = tokenizer.transform(test_dataset).filter_by_query_id(test_query_ids_np)

You can get the user and item mapping and inverse mapping as follows

In [ ]:
print(tokenizer.query_id_encoder.mapping, tokenizer.query_id_encoder.inverse_mapping)
print(tokenizer.item_id_encoder.mapping, tokenizer.item_id_encoder.inverse_mapping)

In [ ]:
train_dataloader = DataLoader(
    dataset=SasRecTrainingDataset(
        sequential_train_dataset,
        max_sequence_length=10,
    ),
    batch_size=3,
    shuffle=True,
    num_workers=4,
    pin_memory=True,
)

In [ ]:
for batch in train_dataloader:
    print(batch.features["item_id_seq"])

    print(batch.query_id)
    print(batch.features["user_cat_not_seq"])
    print(batch.features["user_cat_list_not_seq"])
    break

## Train model
### Create SASRec model instance and run the training stage using lightning
After each epoch validation metrics are shown. You can change the list of validation metrics in ValidationMetricsCallback
The model is determined to be the best and is saved if the metric updates its maximum during validation (see the ModelCheckpoint)

In [ ]:
MAX_SEQ_LEN = 200
BATCH_SIZE = 512
NUM_WORKERS = 4

model = SasRec(
    tensor_schema,
    block_count=2,
    head_count=2,
    max_seq_len=MAX_SEQ_LEN,
    hidden_size=300,
    dropout_rate=0.5,
    optimizer_factory=FatOptimizerFactory(learning_rate=0.001),
)

csv_logger = CSVLogger(save_dir=".logs/train", name="SASRec_example")

checkpoint_callback = ModelCheckpoint(
    dirpath=".checkpoints",
    save_top_k=1,
    verbose=True,
    # if you use multiple dataloaders, then add the serial number of the dataloader to the suffix of the metric name.
    # For example,"recall@10/dataloader_idx_0"
    monitor="recall@10",
    mode="max",
)

validation_metrics_callback = ValidationMetricsCallback(
    metrics=["map", "ndcg", "recall"],
    ks=[1, 5, 10, 20],
    item_count=train_dataset.item_count,
    postprocessors=[RemoveSeenItems(sequential_validation_dataset)]
)

trainer = L.Trainer(
    max_epochs=100,
    callbacks=[checkpoint_callback, validation_metrics_callback],
    logger=csv_logger,
)

train_dataloader = DataLoader(
    dataset=SasRecTrainingDataset(
        sequential_train_dataset,
        max_sequence_length=MAX_SEQ_LEN,
    ),
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=4,
    pin_memory=True,
)

validation_dataloader = DataLoader(
    dataset=SasRecValidationDataset(
        sequential_validation_dataset,
        sequential_validation_gt,
        sequential_train_dataset,
        max_sequence_length=MAX_SEQ_LEN,
    ),
    batch_size=BATCH_SIZE,
    num_workers=NUM_WORKERS,
    pin_memory=True,
)

trainer.fit(
    model,
    train_dataloaders=train_dataloader,
    val_dataloaders=validation_dataloader,
)

The path to the best model is saved inside checkpoint_callback

In [ ]:
best_model = SasRec.load_from_checkpoint(checkpoint_callback.best_model_path)

## Inference stage
### Prepare Dataloader and logger

In [ ]:
prediction_dataloader = DataLoader(
    dataset=SasRecPredictionDataset(
        sequential_test_dataset,
        max_sequence_length=MAX_SEQ_LEN,
    ),
    batch_size=BATCH_SIZE,
    num_workers=NUM_WORKERS,
    pin_memory=True,
)

csv_logger = CSVLogger(save_dir=".logs/test", name="SASRec_example")

### Run inference
You can get the recommendations in three formats: PySpark DataFrame, Pandas DataFrame, PyTorch tensors. Each of the types corresponds a callback
You can filter the results using postprocessors strategy. For example the RemoveSeenItems postprocessor is filtering out the items that already have been seen in test dataset
You don't need to use all three callbacks. This is shown only for example

Also, you can get user embeddings, that were used to perform predictions, using `get_query_embedding` method inside SasRecModel or `QueryEmbeddingsPredictionCallback` for lightning module.

In [ ]:
TOPK = [1, 10, 20, 100]

postprocessors = [RemoveSeenItems(sequential_test_dataset)]

spark_prediction_callback = SparkPredictionCallback(
    spark_session=spark_session,
    top_k=max(TOPK),
    query_column="user_id",
    item_column="item_id",
    rating_column="score",
    postprocessors=postprocessors,
)

pandas_prediction_callback = PandasPredictionCallback(
    top_k=max(TOPK),
    query_column="user_id",
    item_column="item_id",
    rating_column="score",
    postprocessors=postprocessors,
)

torch_prediction_callback = TorchPredictionCallback(
    top_k=max(TOPK),
    postprocessors=postprocessors,
)

query_embeddings_callback = QueryEmbeddingsPredictionCallback()

trainer = L.Trainer(
    callbacks=[
        spark_prediction_callback,
        pandas_prediction_callback,
        torch_prediction_callback,
        query_embeddings_callback
    ], 
    logger=csv_logger, 
    inference_mode=True
)
trainer.predict(best_model, dataloaders=prediction_dataloader, return_predictions=False)

spark_res = spark_prediction_callback.get_result()
pandas_res = pandas_prediction_callback.get_result()
torch_user_ids, torch_item_ids, torch_scores = torch_prediction_callback.get_result()
user_embeddings = query_embeddings_callback.get_result()

In [ ]:
spark_res.show()

In [ ]:
pandas_res

In [ ]:
print(torch_user_ids[0], torch_item_ids[0], torch_scores[0])

Suppose we want to get the recomendations in PySpark format. 
Let's get the inverse representation of labels using inverse_transform method.

Note that the reverse representation can only be obtained for PySpark and Pandas formats. When working with PyTorch tensors, the reverse representation must be done manually

In [ ]:
recommendations = tokenizer.query_and_item_id_encoder.inverse_transform(spark_res)

In [ ]:
recommendations.show()

### Calculating metrics

In [ ]:
init_args = {"query_column": "user_id", "rating_column": "score"}

In [ ]:
result_metrics = OfflineMetrics(
    [Recall(TOPK), Precision(TOPK), MAP(TOPK), NDCG(TOPK), MRR(TOPK), HitRate(TOPK)], **init_args
)(recommendations.toPandas(), raw_test_gt)

In [ ]:
metrics_to_df(result_metrics)

### User embeddings

Got 6040 x 300 user embeddings, because among all 12 batches: 

11 batches contains 512 samples

1 batch contains 408 left samples

11 * 512 + 408 == 6040

In [ ]:
user_embeddings

In [ ]:
user_embeddings.shape

You can access user embeddings directly with `SasRecModel` class

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

core_model = SasRecModel(
    tensor_schema,
    num_blocks=2,
    num_heads=2,
    max_len=MAX_SEQ_LEN,
    hidden_size=300,
    dropout=0.5
)
core_model.eval()
core_model = core_model.to(device)

# Get first batch of data 
data = next(iter(prediction_dataloader))
tensor_map, padding_mask = data.features, data.padding_mask

# Ensure everything is on the same device
padding_mask = padding_mask.to(device)
tensor_map["item_id_seq"] = tensor_map["item_id_seq"].to(device)

# Get user embeddings
user_embeddings_batch = core_model.get_query_embeddings(tensor_map, padding_mask)
user_embeddings_batch

In [ ]:
user_embeddings_batch.shape

### Item embeddings

`get_all_embeddings()` method in transformers can be used to get copies of all embeddings that are presented in model as a dict.

In [ ]:
all_embeddings = best_model.get_all_embeddings()
all_embeddings

You can access item embeddings from this dictionary

In [ ]:
item_embeddings = all_embeddings["item_embedding"]
item_embeddings

Item embeddings shape is (N_ITEMS, HIDDEN_SIZE)

In [ ]:
item_embeddings.shape

Ensure we got correct dimension and ensure we got the copy of tensor

In [ ]:
assert item_embeddings.shape[0] == len(tokenizer.item_id_encoder.mapping["item_id"])
assert id(item_embeddings) != id(best_model._model.item_embedder.item_emb.weight.data)

For example we observe one new item id in our training data. We can easily expand our item embedder by one element

In order to expand item embeddings by new size `set_item_embeddings_by_size` method is applied

In [ ]:
best_model.set_item_embeddings_by_size(item_embeddings.shape[0] + 1)

Now our new item embeddings have one extra embedding

In [ ]:
new_size = best_model.get_all_embeddings()["item_embedding"].shape[0]
old_size = item_embeddings.shape[0]

assert new_size == old_size + 1

Alternatively, we can pass our item embeddings that replace the existing ones by calling `set_item_embeddings_by_tensor`.

If tensor contains new items, they will be added to item embedder.

In [ ]:
new_embeddings_weights = torch.rand((new_size + 1, 300))    # randint used for example only

best_model.set_item_embeddings_by_tensor(new_embeddings_weights)

At the moment we expanded our item embeddings by one more item and replace weights by passing `new_embeddings_weights`

In [ ]:
old_size = new_size
new_size = best_model.get_all_embeddings()["item_embedding"].shape[0]

assert new_size == old_size + 1

Similarly, we can append tensor for only new items with no replace for existing by calling `append_item_embeddings`

In [ ]:
new_item_weights = torch.rand((1, 300))    # randint used for example only

best_model.append_item_embeddings(new_item_weights)

We passed one new example and its weights to item embeddings, thus expanded our vocabulary by one item again

In [ ]:
old_size = new_size
new_size = best_model.get_all_embeddings()["item_embedding"].shape[0]

assert new_size == old_size + 1

## Example of launching an inference for a single user without using a trainer (in order to speed up)
An example for the production of an online script

Let's assume that the user's sequence consisted of a sequence of items [1, 2, 3, 4, 5]. 
Сreate a padding mask corresponding to the sequence of items.

It is important to take only the latest MAX_SEQ_LEN or less items.

In [ ]:
item_sequence = torch.arange(1, 5).unsqueeze(0)[:, -MAX_SEQ_LEN:]
padding_mask = torch.ones_like(item_sequence, dtype=torch.bool)
sequence_item_count = item_sequence.shape[1]

### Wrapping created tensors in the SasRecPredictionBatch entity

In [ ]:
batch = SasRecPredictionBatch(
    query_id=torch.arange(0, item_sequence.shape[0], 1).long(),
    padding_mask=padding_mask.bool(),
    features={ITEM_FEATURE_NAME: item_sequence.long()}
)

### Run predict step of the SasRec and get scores from the model

In [ ]:
with torch.no_grad():
    scores = best_model.predict_step(batch, 0)
scores

### Getting three items with the highest score

In [ ]:
torch.topk(scores, k=3).indices